In [4]:
from scripts.convert import convert_to_openai_function, convert_to_openai_tool
from pydantic.v1 import BaseModel, Field
from datetime import datetime
from pprint import pp

class AddExpense(BaseModel):
    description: str = None
    net_expense: float = None
    gross_expense: float = None
    tax_rate: float = None
    date: datetime = None
    

class ReportMissing(BaseModel):
    missing_fields: list[str]
    
class ReportResult(BaseModel):
    result: str
    
class GetCurrentDate(BaseModel):
    input: str
    
db_tool = convert_to_openai_tool(AddExpense)
missing_info = convert_to_openai_tool(ReportMissing)
report_tool = convert_to_openai_tool(ReportResult)
get_current_date = convert_to_openai_tool(GetCurrentDate)

add_expense
report_missing
report_result
get_current_date


In [32]:
get_current_date

{'type': 'function',
 'function': {'name': 'get_current_date',
  'description': '',
  'parameters': {'type': 'object',
   'properties': {'input': {'type': 'string'}},
   'required': ['input']}}}

In [33]:
AddExpense.schema()

{'title': 'AddExpense',
 'type': 'object',
 'properties': {'description': {'title': 'Description', 'type': 'string'},
  'net_expense': {'title': 'Net Expense', 'type': 'number'},
  'gross_expense': {'title': 'Gross Expense', 'type': 'number'},
  'tax_rate': {'title': 'Tax Rate', 'type': 'number'},
  'date': {'title': 'Date', 'type': 'string', 'format': 'date-time'}}}

In [1]:
from openai import OpenAI
import os

model_name="gpt-3.5-turbo-0125"

client = OpenAI()

In [17]:
system_msg = """You are in charge of completing tasks using a set of specialized tools. Each tool performs a specific function. Here's how to proceed:

Identify the Task: Understand the task that needs to be done.
Select the Tool: Choose the tool best suited for this task based on its unique capability.
Prepare the Input: Think step by step if the tool parameters are known and rational. You may need the output of one tool as the input for another.
"""

In [21]:
messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": "I have spend 5.99 $ for a coffee today, please track my expense. The tax rate is 0.19."}
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=[db_tool, missing_info, report_tool, get_current_date],
    tool_choice="auto"
)

In [35]:
response.choices[0].message

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

In [25]:
tool_calls

[ChatCompletionMessageToolCall(id='call_L3K2NkHaIyrcDcstnXWbMK41', function=Function(arguments='{"description":"Coffee expense","gross_expense":5.99,"tax_rate":0.19}', name='add_expense'), type='function')]

In [24]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_L3K2NkHaIyrcDcstnXWbMK41', function=Function(arguments='{"description":"Coffee expense","gross_expense":5.99,"tax_rate":0.19}', name='add_expense'), type='function')])

In [40]:
import json

args_str = response_message.tool_calls[0].function.arguments
args_str = '{"description":"Coffee expense","gross_expense":5.99,"tax_rate":0.19, "date":"2022-10-10"}'
args = json.loads(args_str)
args

{'description': 'Coffee expense',
 'gross_expense': 5.99,
 'tax_rate': 0.19,
 'date': '2022-10-10'}

In [25]:
tool_call = tool_calls[0]
tool_call_message = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": "The net expense and gross expense should not be the same if a tax rate is provided. Please provide the correct net expense or gross expense."
                }

In [27]:
response_2 = client.chat.completions.create(
    model=model_name,
    messages=messages + [response_message, tool_call_message],
    tools=[db_tool, missing_info, report_tool],
    tool_choice="auto"
)

response_2.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Zz2CrJhDzIss09OrPAXL4uZ3', function=Function(arguments='{"description":"Coffee expense","net_expense":5.99,"gross_expense":7.13,"tax_rate":0.19,"date":"2023-10-16"}', name='add_expense'), type='function')])

In [28]:
response_2

ChatCompletion(id='chatcmpl-8zSsO4UMkcpFZlTQwVEPEaLgfJFsz', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Zz2CrJhDzIss09OrPAXL4uZ3', function=Function(arguments='{"description":"Coffee expense","net_expense":5.99,"gross_expense":7.13,"tax_rate":0.19,"date":"2023-10-16"}', name='add_expense'), type='function')]))], created=1709659700, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_2b778c6b35', usage=CompletionUsage(completion_tokens=48, prompt_tokens=361, total_tokens=409))

In [29]:
response

ChatCompletion(id='chatcmpl-8zSqIawSA9OtMsLoqylbdehFtOYQI', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7qC42fEXA3EIinIBNQEyuFgF', function=Function(arguments='{"description":"Coffee expense","net_expense":5.99,"gross_expense":0,"tax_rate":0.19,"date":"2023-10-16"}', name='add_expense'), type='function')]))], created=1709659570, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_2b778c6b35', usage=CompletionUsage(completion_tokens=46, prompt_tokens=278, total_tokens=324))

In [35]:
tools = [db_tool, missing_info, report_tool]

tools

[{'type': 'function',
  'function': {'name': 'add_expense',
   'description': '',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string'},
     'net_expense': {'type': 'number'},
     'gross_expense': {'type': 'number'},
     'tax_rate': {'type': 'number'},
     'date': {'type': 'string', 'format': 'date-time'}},
    'required': ['description',
     'net_expense',
     'gross_expense',
     'tax_rate',
     'date']}}},
 {'type': 'function',
  'function': {'name': 'report_missing',
   'description': '',
   'parameters': {'type': 'object',
    'properties': {'missing_fields': {'type': 'array',
      'items': {'type': 'string'}}},
    'required': ['missing_fields']}}},
 {'type': 'function',
  'function': {'name': 'report_result',
   'description': '',
   'parameters': {'type': 'object',
    'properties': {'result': {'type': 'string'}},
    'required': ['result']}}}]

In [50]:
check_prompt = """You are responsible to verify the action of an assistant based on the the chat history of a user and the assistant. Your role is to verify the assistant's actions and report any mistakes.

The assistant should complete a task given by the user using a set of specialized tools. The assistant tends to hallucinate and make mistakes. He might pass parameters to a tool that were not mentioned in the chat_history or use the wrong tool for the task. 
Verify if the assistant choice the correct tool for the task and if the tool was used correctly.

Check each input parameter the assistant used and verify if it was mentioned in the chat history.
"""

user_message_tmpl = "Here is the chat history: {chat_history}\nHere are the tools the assistant can choose from: {tools}"
assistant_action_tmpl = "The assistant used the {tool} tool with the following parameters: {parameters}"




In [ ]:
example_response = {"verified_action":False, "reason": "The date of the expense is not mentioned in the chat history. Instead the assistant should have used get_current_date tool to get the current date. The gross expense is not passed correctly the assitant could calculate it using the net_expense and the tax_rate."}


import json
response_check.choices[0].message.tool_calls[0].function.arguments = json.dumps(example_response)
example_response = response_check.choices[0].message.tool_calls[0].function.arguments

In [51]:
history = [{"role": "user", "content": "I have spend 5.99 $ for a coffee today, please track my expense. The tax rate is 0.19."}]
user_check_message = user_message_tmpl.format(chat_history=history, tools=tools)
assistant_action = assistant_action_tmpl.format(tool=tool_calls[0].function.name, parameters= tool_calls[0].function.arguments)


In [52]:
class validate_action(BaseModel):
    verified_action: bool
    reason: str
    
    
validate_action_tool = convert_to_openai_tool(validate_action)


response_check = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "system", "content": check_prompt}, {"role": "user", "content": user_check_message, "name": "example_user"}, {"role": "user", "content": assistant_action, "name": "example_user"}, {"role": "assistant", "content": example_response, "name": "example_assistant"}, ],
    tools=[validate_action_tool],
    tool_choice="auto"
)

In [55]:
assistant_action

'The assistant used the add_expense tool with the following parameters: {"description":"Coffee expense","net_expense":5.99,"gross_expense":null,"tax_rate":0.19,"date":"2023-10-05T00:00:00Z"}'

In [57]:
response_check.choices[0].message.tool_calls[0].function.arguments

'{"verified_action":true,"reason":"The assistant correctly used the add_expense tool with the provided parameters based on the user\'s request to track the coffee expense with the tax rate."}'

In [58]:
example_response = {"verified_action":False, "reason": "The date of the expense is not mentioned in the chat history. Instead the assistant should have used get_current_date tool to get the current date. The gross expense is not passed correctly the assitant could calculate it using the net_expense and the tax_rate."}


import json
response_check.choices[0].message.tool_calls[0].function.arguments = json.dumps(example_response)
example_response = response_check.choices[0].message.tool_calls[0].function.arguments

In [31]:
import re
string = "AddDBEntry"

def split_on_capital_case(s):
    """Split a string on capital case letters.
    
    If multiple capital case letters are together, they are considered as one word.
    """
    matches = re.findall(r'[A-Z][a-z]*', s)
    result = []
    temp = ''
    for match in matches:
        if len(match) == 1:
            temp += match
        else:
            if temp:
                result.append(temp)
                temp = ''
            result.append(match)
    if temp:
        result.append(temp)
    return result

split_on_capital_case(string)

['Add', 'D', 'B', 'Entry']


['Add', 'DB', 'Entry']